In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('Vol3ProjectData.csv',sep='\t')

/home/connor/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
display(df)

,user_id,concept_id,prioritized,updated_timestamp
0,3387028845212925,5988,0,1592605795168
1,3387028845212925,2044,0,1592605082266
2,3387028845212925,5264,0,1592605533164
3,3387028845212925,143,0,1592605082270
4,3387028845212925,5405,0,1592605533167
...,...,...,...,...
5411596,3843594269610093,49064,0,1634770540638
5411597,3843594269610093,1799,0,1635113923999
5411598,3843594269610093,2095,0,1634770785183
5411599,3843594269610093,4976,0,1635113924000


In [3]:
#Change all the values in the prioritized column to strings.
df['prioritized'] = df['prioritized'].astype(str)
df = df[df['prioritized'] != "\\N"]

#Change all the timestamps to ints.
df = df[df['updated_timestamp'] != "\\N"]
df['updated_timestamp'] = df['updated_timestamp'].astype(np.int64)

#Change all the user IDs to strings.
df = df[df['user_id'] != "\\N"]
df['user_id'] = df['user_id'].astype(str)

#Change all the concept IDs to strings.
df = df[df['concept_id'] != "\\N"]
df['concept_id'] = df['concept_id'].astype(str)

# Add the starred and mistaken rows.
# Starred words have priority 2 or 3; mistaken words have priority 1 or 3.
df['starred'] = df['prioritized'].astype(np.int64) > 1
df['mistaken'] = df['prioritized'].astype(np.int64) % 2 == 1

#reset the indices
df = df.reset_index(drop = True)

display(df)

,user_id,concept_id,prioritized,updated_timestamp,starred,mistaken
0,3387028845212925,5988,0,1592605795168,False,False
1,3387028845212925,2044,0,1592605082266,False,False
2,3387028845212925,5264,0,1592605533164,False,False
3,3387028845212925,143,0,1592605082270,False,False
4,3387028845212925,5405,0,1592605533167,False,False
...,...,...,...,...,...,...
5375193,3843594269610093,49064,0,1634770540638,False,False
5375194,3843594269610093,1799,0,1635113923999,False,False
5375195,3843594269610093,2095,0,1634770785183,False,False
5375196,3843594269610093,4976,0,1635113924000,False,False


In [ ]:
#Create the words_studied column:

# Cast the user IDs to ints for faster sorting.
df['user_id'] = df['user_id'].astype(np.int64)

#get all the unique values in user_id
unique = df['user_id'].unique()
#create a list of zeros the same length of the dataframe
words_studied = np.zeros(len(df['user_id']))
#cylce through each unique user_id (this takes about 5 min to run)
for i in unique:
    #define a dataframe where the user IDs match the value we are checking in the unique IDs
    sf = df.loc[df['user_id'] == i]
    #sort that dataframe by the timestamps
    sf = sf.sort_values('updated_timestamp')
    #gather those indices
    sf = sf.index
    #cycle through those indices and add the number of word that it is that they have learned at the index in words_studied
    for i in range(1,len(sf)+1):
        words_studied[sf[i-1]] = i
#create the column and add the list to that column
df['words_studied'] = words_studied.astype(np.int32)

# Cast the user IDs back to strings.
df['user_id'] = df['user_id'].astype(str)

display(df)

In [5]:
# Get the set of user_ids for users who have starred at least one word.
starred = df.loc[df['starred'] == True]
user_ids_who_star = set(starred['user_id'])

# Get all the rows of each user who has starred a word.
users_who_star = df.loc[df['user_id'].isin(user_ids_who_star)]
display(users_who_star)

,user_id,concept_id,prioritized,updated_timestamp,starred,mistaken,words_studied
462,3391866971749096,476,0,1612281478803,False,False,3
463,3391866971749096,2395,2,1631583836797,True,False,17
464,3391866971749096,2044,0,1631583871428,False,False,18
465,3391866971749096,1989,0,1631584322053,False,False,32
466,3391866971749096,56392,0,1612281493497,False,False,5
...,...,...,...,...,...,...,...
5375151,3839423983477931,4204,2,1630446541480,True,False,2
5375152,3839423983477931,4218,0,1630446540749,False,False,1
5375153,3839423983477931,6264,0,1630446629508,False,False,6
5375154,3839423983477931,2992,0,1630446629508,False,False,7


In [6]:
# Save the cleaned data to a csv.
df.to_csv('Vol3CleanedData.csv')
users_who_star.to_csv('Vol3StarredData.csv')